In [ ]:
!pip install selenium
!apt-get update

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

In [ ]:
# URL de consulta (octubre 2024)
url = 'https://www.timeanddate.com/weather/mexico/mexico-city/historic?month=10&year=2024'

# Solicitud inicial con requests
response = requests.get(url)
if response.status_code == 200:
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
# Extraer los links de días disponibles
dia = soup.find('div', attrs={'class': 'weatherLinks'})
days = dia.find_all('a')
links = [day.text for day in days]

In [ ]:
# Iniciar Selenium
driver = web_driver()
driver.get(url)

In [ ]:
# Lista para almacenar los datos
data_list = []

# Loop principal con respaldo cada 10 iteraciones
for idx, dia in enumerate(links, start=1):
    try:
        # Encontrar y hacer clic en el enlace del día
        enlace = driver.find_element(By.XPATH, f'//a[text()='{dia}']')
        enlace.click()
        time.sleep(3)  # esperar a que cargue la tabla

        # Obtener HTML de la tabla
        html_table = driver.find_element(By.ID, "wt-his").get_attribute("outerHTML")
        soup = BeautifulSoup(html_table, 'html.parser')
        tabla = soup.find('table')

        # Extraer filas
        for fila in tabla.find_all('tr')[1:]:
            celdas = [celda.text for celda in fila.find_all('td')]
            data_list.append(celdas)

        # Guardar copia de seguridad cada 10 iteraciones
        if idx % 10 == 0:
            backup_name = f'weather_backup_{idx}.xlsx'
            pd.DataFrame(data_list).to_excel(backup_name, index=False)
            print(f'Copia de seguridad guardada: {backup_name}')

    except Exception as e:
        print(f'Error en el día {dia}: {e}')

In [ ]:
# Guardado final
pd.DataFrame(data_list).to_excel('weather.xlsx', index=False)
print('Archivo final guardado: weather.xlsx')